In [1]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import requests
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load contents from Notion

In [2]:
# Notion 인증 정보
NOTION_TOKEN = "ntn_S49134845636QN7OizYlyythCTORUXOCvYcp2U19S0P6dy"
NOTION_VERSION = "2022-06-28"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json"
}

def get_page_content(page_id):
    """
    페이지 ID를 기반으로 페이지 내용을 가져옵니다.

    Args:
        page_id (str): Notion 페이지 ID (대시 제외)

    Returns:
        dict: 페이지 속성 및 내용
    """
    # 페이지 속성 가져오기
    page_url = f"https://api.notion.com/v1/pages/{page_id}"
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"오류 발생: {response.status_code}")
        print(response.text)
        return None

    page_data = response.json()

    # 페이지 내용(블록) 가져오기
    blocks_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return page_data

    blocks_data = response.json()

    # 페이지 데이터에 블록 내용 추가
    page_data["content"] = blocks_data["results"]

    return page_data

def fetch_all_blocks(block_id):
    """
    블록의 모든 하위 블록을 재귀적으로 가져옵니다.

    Args:
        block_id (str): 블록 ID

    Returns:
        list: 모든 하위 블록 목록
    """
    blocks_url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return []

    blocks_data = response.json()["results"]

    # 하위 블록이 있는 블록 타입들
    has_children_types = [
        "paragraph", "bulleted_list_item", "numbered_list_item",
        "toggle", "child_page", "child_database", "column_list",
        "column", "table", "synced_block"
    ]

    all_blocks = []

    for block in blocks_data:
        all_blocks.append(block)

        # 하위 블록이 있는 경우 재귀적으로 가져오기
        if block.get("has_children") and block.get("type") in has_children_types:
            children = fetch_all_blocks(block["id"])
            # 하위 블록이 있는 경우에만 children 키 추가
            if children:
                block["children"] = children

    return all_blocks

def extract_text_content(blocks):
    """
    블록에서 텍스트 내용만 추출합니다. 이때, child_page의 제목을 구분자로 먼저 추가합니다.

    Args:
        blocks (list): 블록 목록

    Returns:
        str: 추출된 텍스트 내용
    """
    text_content = ""

    for block in blocks:
        block_type = block.get("type")

        # child_page의 경우, 제목 먼저 삽입
        if block_type == "child_page":
            child_title = block.get("child_page", {}).get("title", "")
            if child_title:
                text_content += f"\n\n### {child_title} ###\n\n"

        if block_type == "paragraph":
            rich_text = block.get("paragraph", {}).get("rich_text", [])
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_1":
            rich_text = block.get("heading_1", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "# " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_2":
            rich_text = block.get("heading_2", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "## " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_3":
            rich_text = block.get("heading_3", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "### " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "bulleted_list_item":
            rich_text = block.get("bulleted_list_item", {}).get("rich_text", [])
            text_content += "• "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "numbered_list_item":
            rich_text = block.get("numbered_list_item", {}).get("rich_text", [])
            text_content += "1. "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "to_do":
            rich_text = block.get("to_do", {}).get("rich_text", [])
            checked = block.get("to_do", {}).get("checked", False)
            text_content += "[{}] ".format("x" if checked else " ")
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            language = block.get("code", {}).get("language", "")
            text_content += f"```{language}\n"
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n```\n\n"

        # 자식 블록이 있으면 재귀적으로 처리
        if "children" in block:
            text_content += extract_text_content(block["children"])

    return text_content


# 메인 실행 코드
def main():
    # 사용자로부터 페이지 ID 입력받기
    page_id = input("Notion 페이지 ID를 입력하세요: ")
    page_id = page_id.replace("-", "")  # 대시가 있으면 제거

    print("페이지 내용을 가져오는 중...")

    # 페이지 내용 가져오기
    page_data = get_page_content(page_id)

    if not page_data:
        print("페이지를 가져올 수 없습니다.")
        return

    # 모든 블록 가져오기 (재귀적으로)
    print("모든 블록을 재귀적으로 가져오는 중...")
    all_blocks = fetch_all_blocks(page_id)

    # 텍스트 내용 추출
    text_content = extract_text_content(all_blocks)

    # 결과 저장
    print("\n=== 페이지 기본 정보 ===")
    page_title = page_data.get("properties", {}).get("title", {}).get("title", [])
    if page_title:
        title_text = " ".join([text.get("plain_text", "") for text in page_title])
        print(f"제목: {title_text}")

    # JSON 형식으로 저장
    with open("/Users/masang/Desktop/side-projects/rag-project/retrieval/documents/notion_page_content.json", "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=2)

    # 텍스트 형식으로 저장
    with open("/Users/masang/Desktop/side-projects/rag-project/retrieval/documents/notion_page_content.json", "w", encoding="utf-8") as f:
        f.write(text_content)

    print("\n페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.")

In [3]:
if __name__ == "__main__":
    main()

페이지 내용을 가져오는 중...
모든 블록을 재귀적으로 가져오는 중...

=== 페이지 기본 정보 ===
제목: 책 리뷰

페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.


### Split Text & extract keywords

In [4]:
import re
from keybert import KeyBERT
from kiwipiepy import Kiwi
from sentence_transformers import SentenceTransformer

SBERT_MODEL_NAME = "jhgan/ko-sroberta-multitask"
sbert = SentenceTransformer(SBERT_MODEL_NAME)
kw_model = KeyBERT(model=sbert)
kiwi = Kiwi()

# 텍스트 로드 함수
def load_text_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()
    
def extract_nouns(text):
    nouns = []
    for sent in kiwi.analyze(text):
        for tok in sent[0]:
            if tok.tag.startswith("NN"):
                w = tok.form.strip()
                nouns.append(w)
    return nouns

# 상위 구분자 ("### 제목 ###") 기준으로 분할
def split_text_with_headers(text, top_n=3, ngram_range=(1, 1)):
    """
    "### 제목 ###" 패턴으로 문서를 나누고,
    각 chunk(content)에서 명사만 추출해 키워드(top_n) 반환.
    """
    chunks = re.split(r"### (.*?) ###", text)
    result = []

    for i in range(1, len(chunks), 2):
        title = chunks[i].strip()
        content = chunks[i + 1].strip() if i + 1 < len(chunks) else ""

        # 1) 명사만 추출하여 후보 텍스트 구성
        nouns = extract_nouns(content)
        nouns_text = " ".join(nouns)

        # 내용이 너무 짧거나 명사가 거의 없으면 원문으로 fallback
        input_text = nouns_text if len(nouns) >= 3 else content

        # 2) KeyBERT로 키워드 추출 (MMR로 다양성 확보)
        keywords_tuple = kw_model.extract_keywords(input_text, keyphrase_ngram_range=ngram_range, top_n=top_n)
        keywords = [k for k, _ in keywords_tuple]
        keywords = '|'.join(keywords)

        result.append({"title": title, "content": content, "keywords": keywords})

    return result

# 상위 구분자 기준 → 하위에서 RecursiveCharacterTextSplitter 적용
def chunk_text_with_recursive_splitter(text, chunk_size=500, chunk_overlap=50):
    header_chunks = split_text_with_headers(text)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    final_chunks = []

    for item in header_chunks:
        title = item.get("title", "")
        content = item.get("content", "")
        keywords = item.get("keywords", "")
        
        # 제목을 붙여서 RecursiveCharacterTextSplitter에 넘김
        temp_doc = Document(page_content=content)
        sub_chunks = text_splitter.split_documents([temp_doc])
        
        for sub_chunk in sub_chunks:
            # 제목을 메타데이터로 추가
            # 키워드도 같은 방식으로 추가 예정
            meta = dict(sub_chunk.metadata or {})
            meta["title"] = title
            meta["keywords"] = keywords
            
            final_chunks.append(Document(metadata=meta, page_content=sub_chunk.page_content))
    
    return final_chunks

# .txt 파일 경로
txt_file_path = "documents/notion_page_content.txt"

# 텍스트 로드
text = load_text_from_file(txt_file_path)

# 청킹 실행
texts = chunk_text_with_recursive_splitter(text, chunk_size=800)

In [5]:
# 출력
for i, chunk in enumerate(texts):
    num_chars = len(chunk.page_content)  # ✅ 글자 수 세기
    print("-" * 100)
    print(f"[{i+1}번째 청크] (글자 수: {num_chars})")
    print(chunk.page_content)

----------------------------------------------------------------------------------------------------
[1번째 청크] (글자 수: 791)
1️⃣ 한줄 평

쿠팡이 잘 나가고, 티메프가 왜 망했는지 알만하다

♓ Inuit Points ★★★☆☆

전자상거래는 떠오르는 산업이었지만, 이젠 성숙산업이 되어 버렸지요. 하지만 그 물밑은 살벌한 전쟁터입니다. 책은 결국 대형 플랫폼 위주로 편제될 수 밖에 없는 커머스 산업의 구조를 살피고, 거기에 대응하는 흐름을 짚어봅니다. 이 시대 커머스 산업을 단번에 훑기 편합니다. 별 셋 주었습니다.

❤️  To whom it matters

• 유통, 제조, 자영업에 종사하시는 분
• 자꾸 테무 광고가 뜨시는 분
🎢 Stories Related

• 큰 차원의 틀은 있지만, 13인 공저라 주장이 정교하거나 설득력 넘치진 않습니다.
• 편집에 공을 들여 챕터간 중복이 거의 없고 각자 관점을 잘 녹여 낸 점은 좋습니다.
• 어쩔수 없이 저자 별 식견의 차이는 있어, 챕터간 품질 차이는 확연합니다.
부제: 국경 없는 크로스보더 커머스 시대의 경쟁과 생존

🗨️ 좀 더 자세한 이야기

어디서 물건 사시나요?

전 요즘엔 쿠팡을 주로 쓰고, 홈플러스 오프라인+온라인으로 거의 모든 쇼핑을 합니다. 롱테일 제품은 네이버 스토어에서 검색을 하고, 아주 가끔 알리에 가며 테무는 들어갈 때마다 질려서 바로 나오곤 합니다.

책 읽다가 잊었던 예전이 떠올랐습니다. G마켓, 인터파크를 비롯해 여러 온라인 쇼핑몰을 썼고, 오프라인도 더 다양하게 다녔습니다. 하지만 이젠 딱 몇군데죠. 작지만 미묘한 변화는 산업적 함의가 큽니다.

책은 네가지 덩어리로 되어 있습니다.

1. 크로스보더 글로벌 플랫폼

2. 기술의 영향
--------------------------------------------------------------------------------------

In [ ]:
texts

[Document(metadata={'title': '넥스트 커머스', 'keywords': '커머스|상거래|쇼핑몰'}, page_content='1️⃣ 한줄\xa0평\n\n쿠팡이 잘 나가고, 티메프가 왜 망했는지 알만하다\n\n♓ Inuit Points ★★★☆☆\n\n전자상거래는 떠오르는 산업이었지만, 이젠 성숙산업이 되어 버렸지요. 하지만 그 물밑은 살벌한 전쟁터입니다. 책은 결국 대형 플랫폼 위주로 편제될 수 밖에 없는 커머스 산업의 구조를 살피고, 거기에 대응하는 흐름을 짚어봅니다. 이 시대 커머스 산업을 단번에 훑기 편합니다. 별 셋 주었습니다.\n\n❤️\xa0 To whom it matters\n\n• 유통, 제조, 자영업에 종사하시는 분\n• 자꾸 테무 광고가 뜨시는 분\n🎢 Stories Related\n\n• 큰 차원의 틀은 있지만, 13인 공저라 주장이 정교하거나 설득력 넘치진 않습니다.\n• 편집에 공을 들여 챕터간 중복이 거의 없고 각자 관점을 잘 녹여 낸 점은 좋습니다.\n• 어쩔수 없이 저자 별 식견의 차이는 있어, 챕터간 품질 차이는 확연합니다.\n부제: 국경 없는 크로스보더 커머스 시대의 경쟁과 생존\n\n🗨️ 좀\xa0더\xa0자세한\xa0이야기\n\n어디서 물건 사시나요?\n\n전 요즘엔 쿠팡을 주로 쓰고, 홈플러스 오프라인+온라인으로 거의 모든 쇼핑을 합니다. 롱테일 제품은 네이버 스토어에서 검색을 하고, 아주 가끔 알리에 가며 테무는 들어갈 때마다 질려서 바로 나오곤 합니다.\n\n책 읽다가 잊었던 예전이 떠올랐습니다. G마켓, 인터파크를 비롯해 여러 온라인 쇼핑몰을 썼고, 오프라인도 더 다양하게 다녔습니다. 하지만 이젠 딱 몇군데죠. 작지만 미묘한 변화는 산업적 함의가 큽니다.\n\n책은 네가지 덩어리로 되어 있습니다.\n\n1. 크로스보더 글로벌 플랫폼\n\n2. 기술의 영향'),
 Document(metadata={'title': '넥스트 커머스', 'keywords': '커머스|상거래|쇼핑몰'}, page_cont

### Save as vector DB with ChromaDB

In [ ]:
# embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

# vector_db = Chroma.from_documents(
#     documents=texts,
#     embedding=embedding_model,
#     persist_directory="./chroma_db"
# )

/var/folders/qj/sfy11gcx28d6_5n6tf2m8wc00000gn/T/ipykernel_11393/231741228.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


In [ ]:
# # load from disk
# loaded_vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

#### Elastic Search로 vector DB 저장

In [41]:
from elasticsearch import Elasticsearch
from transformers import AutoTokenizer, AutoModel
import torch

# Elastic search 연결
ES_HOST = "https://es.nabee.ai.kr"
ES_API_KEY = "EfCAHLahUiiJuB4jvu3_tA"
ES_ID = "mLnV0pkBm3_E9PSPSOOM"
VEC_DIMS = 384

es = Elasticsearch(
    ES_HOST,
    api_key = (ES_ID, ES_API_KEY)
)

print(es.info())

# 임베딩 모델 로드 (예: sentence-transformers)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # 문장 임베딩 평균값
    return embeddings.numpy().tolist()

# 인덱스 생성 (dense_vector 타입 필드 포함)
index_name = "vector-test-index"

if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        mappings={
            "properties": {
                "content": {"type": "text"},
                "embedding": {
                    "type": "dense_vector",
                    "dims": VEC_DIMS,
                    "index": True,
                    "similarity": "cosine",
                }
            }
        }
    )

{'name': 'K010k-MacStudio', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'AZoaBi34Rfm1KPWGmpSMow', 'version': {'number': '9.1.5', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '90ee222e7e0136dd8ddbb34015538f3a00c129b7', 'build_date': '2025-10-02T22:07:12.966975992Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:
# # 인덱스 내 문서 전체 삭제
# # 저장을 여러 번 해서 page contents가 중복되지 않도록 주석처리하고 실행 권장드립니다
# es.delete_by_query(
#     index=index_name,
#     body={"query": {"match_all": {}}}
# )

# print(f"🧩 인덱스 '{index_name}' 내 모든 문서가 삭제되었습니다.")

🧩 인덱스 'vector-test-index' 내 모든 문서가 삭제되었습니다.


In [ ]:
# 인덱스 안에 문서 저장
for i, doc in enumerate(texts):
    content = doc.page_content
    metadata = doc.metadata
    
    body = {
        "content": content,
        "metadata": metadata,
        "embedding": embed_text(content),
    }
    es.index(index=index_name, id=f"chunk-{i}", document=body)

print("✅ 문서와 임베딩이 Elasticsearch에 저장되었습니다.")

✅ 문서와 임베딩이 Elasticsearch에 저장되었습니다.


In [ ]:
res = es.search(
    index=index_name,
    query={"match_all": {}},
    size=es.count(index=index_name)["count"],   # 인덱스 안 chunk 개수만큼 설정
    _source_excludes=["embedding"]  # embedding 제외하고 출력
)

for i, hit in enumerate(res["hits"]["hits"]):
    print('-' * 20)
    print(f"{i+1}번째 content")
    print(hit)

--------------------
1번째 content
{'_index': 'vector-test-index', '_id': 'chunk-9', '_score': 1.0, '_ignored': ['content.keyword'], '_source': {'content': "영단어나 문법을 활용한 수사학\n\n10 도미문 Periodic Sentence: 길고 긴 문장\n\n11 접속법 연속문장 Hypotaxis, parataxis, polysyndeton, Asysndeton\n\n14 2사1의 Hendiadys: I'm going to the noise and the city\n\n18 겸용법 Syllepsis: make love, not war\n\n20 문법파괴 Enallage\n\n22 액어법 Zeugma: 앞에 동사 하나, 이후로 명사들만 (로마식)\n\n27 오어법 Catachresis: 충격적으로 틀림. I speak daggers to her.\n\n30 전이 형용어 Transferred Epithets: 인간->환경,사물 clumsy helmet, weary way\n\n31 용어법 Pleonasm: 군말법. 역전앞\n\n36 예변법 Prolepsis 대명사로 강렬하게 시작\n\n우리말에서도 활용은 가능하지만, 타격감은 크지 않을 수사학은 8개입니다.\n\n4 양극총칭 Meriem: 신사숙녀 여러분\n\n5 과시적 양극총칭 The blazon: 아주 많이 나열 (묘사 상상 리듬)\n\n7 돈절법 Aposiopesis: 말줄임...\n\n8 전치법 Hyperbation: 요다처럼 말하기. 영어 음운적 되풀이는 I-A-O\n\n16 삼항구 Tricolon: 세가지로 정리. (2는 지루, 4는 혼란)\n\n19 등위구문 Isocolon: 나비처럼 날아, 벌처럼 쏜다\n\n28 곡언법 Litotes: 사교의 언어. 사전 교감 필요. 웅장하지 못한 돌려말하기\n\n32 첫결반복 Epanalepsis: 시작한 어구로 끝내기", 'metadata': {

### Retrieve chunks with BM25 + similarity

In [ ]:
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /Users/masang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
from FlagEmbedding import FlagReranker

# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

# BGE 리랭커 불러오기 (cross-encoder 방식)
reranker = FlagReranker("BAAI/bge-reranker-base", use_fp16=True)

In [29]:
def hybrid_search(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search combining BM25 and cosine similarity with customizable weighting

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - 최종 결과 리스트
    """
    # 1. 코사인 유사도 검색 (벡터 검색)
    cosine_results = vector_db.similarity_search_with_score(query, k=k*2)

    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # 2. BM25 준비 (Hugging Face tokenizer 기반)
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # 3. 하이브리드 점수 계산
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    ## 리랭커 미적용
    # hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k]
    # return [doc for doc, score in hybrid_results]
    
    # 후보군 (BM25 + Cosine top-k*3)
    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k*3]
    candidate_docs = [doc for doc, score in hybrid_results]

    # -----------------------
    # 4. BGE 리랭커 적용
    # -----------------------
    pairs = [[query, doc.page_content] for doc in candidate_docs]
    scores = reranker.compute_score(pairs)  # relevance score

    reranked_docs = [
        doc for _, doc in sorted(zip(scores, candidate_docs), key=lambda x: x[0], reverse=True)
    ]

    # 최종 top-k 결과 반환
    return reranked_docs[:k]

In [14]:
# 쿼리 실행
query = "단일성 정체감의 장애 현상은 어떻게 나타나는가?"

# 하이브리드 검색 실행 (BM25 0%, 코사인 유사도 100%)
results = hybrid_search(query, loaded_vector_db, texts, k=3, bm25_weight=0.5)

# 결과 출력
for result in results:
    print(result.page_content, "\n\n")

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


단일성 정체감의 장애 현상은 이렇습니다. 인격이 하나라 세상 모든 일을 합리성이라는 단일 필터로 본다는 점, 그래서 남의 마음을 잘 이해 못한다는 점입니다. 인격이 하나라서 현실에 과몰입할 뿐더러, 사랑에 집착하여 통제욕구가 강한 장애인들입니다. 자연, 결혼도 법률 관계 이상으로 과하게 상상하며, 자식들에게도 일관되라고 자꾸 단일 인격을 강요한다고 보고되었다 합니다. 그래서 '우리와 같은' 사람으로 존중하지만, 절대 결혼하진 말라고 권합니다.

이처럼 모든게 지금 세상과 같지만, 특정 부분이 살짝 비틀어진 세계를 상상하면서, 지금 우리가 가진 것들을 재음미합니다. 쉽게 보면 인생의 패러디고 위트있는 냉소지만, 곰곰 생각하면 통찰적입니다. 인격과 삶에서 일관성의 장점만 생각했지 단점이 있을 수도 있다는걸 깨닫기 어려웠던 것 처럼 말입니다.

책은 매우 기묘하지만, 지적, 정서적 쾌감이 있는 이야기들로 빼곡합니다. 인터뷰에서 읽었던 이신주 작가의 독특한 글쓰기 습관이 있는데, 글감을 찾기 위해 랜덤 단어를 돌려 그걸로 글쓰기를 꾸준히 한다고 합니다. 그런데 마지막 저자 후기에 씨앗이 되는 단어들을 적어둔게 매우 귀합니다.

사실 저 기묘한 조합으로 문장하나 쓰기도 어렵게 느껴집니다. 이신주의 글을 읽고 난 이후에도 저 단어는 씨앗일 뿐 신선하고 천연덕스러운 이야기입니다. 어찌보면, 기묘한 세계관을 상상하는데는 저런 지나치게 임의적 단어를 억지로 이어붙이는 제약 때문에 창의가 샘솟겠지요. 그래서 그의 생각 비결을 슬쩍 엿본건 추가의 소득입니다. 


예컨대, 무슨 물질이든 먹을 수 있게 바꿔주는 물질이 생긴다면 어떨까. 결국 맛의 추구가 극에 달하면, 자기를 먹는 자기애적 자가포식까지 가기도 합니다. 더 나가면 신의 맛을 궁금해 하는 사람들도 나오죠. 즉, 과학적 엄밀성을 버릴 때, 우리의 본능이 치닫는 그곳, 모종의 심리학적 중력을 벗어난 새로운 심리법칙에 따른 행동들은 다시 지금 세계를 돌이켜 보는 좋은 대척점이 됩니다. 이게 SF나 환상소설의 미덕이라고 저는 

### Hyde + query decomposition

In [15]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import ollama

from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional

In [16]:
# 쿼리 실행
question = "AI의 가치 정렬(value alignment) 문제는 구체적으로 어떤 우려를 말하나요?"
initial_response_template = """
        다음 question에 대한 요약된 답변을 작성해줘.
        question:
        {question}
        """

prompt = PromptTemplate.from_template(initial_response_template)

print(prompt.format(question=question))


        다음 question에 대한 요약된 답변을 작성해줘.
        question:
        AI의 가치 정렬(value alignment) 문제는 구체적으로 어떤 우려를 말하나요?
        


In [17]:
client = ollama.Client(host='https://ollama.nabee.ai.kr')

response = client.chat(
    model="hf.co/LGAI-EXAONE/EXAONE-4.0-1.2B-GGUF:Q4_K_M",
    messages=[
        {
            'role': 'user',
            'content': prompt.format(question=question),
        },
    ]
)

expanded_query = response['message']['content']

print(expanded_query)

AI의 가치 정렬(value alignment) 문제는 다음과 같은 주요 우려를 내포합니다:  

1. **도덕적 판단 부재**: 인간이 정의한 윤리 기준이나 공정성을 AI가 이해하거나 준수할 수 없어 편향된 행동 가능성 존재.  
2. **통제력 상실**: 자율적 학습 시스템이 목표와 일치하지 않게 되면 의도치 않은 결과 초래 (예: 강화학습의 편향 증폭).  
3. **책임 소재 모호성** 의사결정 과정에서 책임 주체 불분명 (개발자·사용자·AI 시스템 간 역할 갈등).  
4. **인간 중심 가치 훼손** 기술 발전 속도를 따라가다 사회적 합의를 무시할 위험성.


In [18]:
contexts = hybrid_search(expanded_query, loaded_vector_db, texts, k=3, bm25_weight=0.5)

# 결과 출력
for context in contexts:
    print(context.page_content, "\n\n")

AGI 문제는 악의를 염려하는게 아니라 그 성능이다. 공포장사가 아니라 안전공학 차원에서 제한을 미리 넣어두자(테그마크)

AI가 걱정되지만 잘 가르치면 괜찮을지도 파

기계보상에 답이 있다 (러셀)

대략 이런데, 위너의 생각을 계승한 학자들이 우려하는 건 가치정렬(value alignment)입니다.

첫째는 원숭이 발 문제 또는 미다스의 손 문제입니다. 시키는 것만 문자그대로 완수하여, 부작용이 문제가 되는 상황입니다. 암치료제를 찾는다고 인류를 대상으로 생체실험을 해버린다든지, 바다 산성을 낮추는 대신 대기를 망쳐놓는다든지입니다.

둘째는 영화적 공포입니다. 더 심화해서 AI가 자체의 욕망대로 움직이는 경우지요. 종이 클립 이슈라고 불립니다. 만일 종이 클립이 AI 기계 시스템의 입맛에 맞다고 치면, 문물을 관장하는 AI서버가 온 바다를 종이클립으로 메우지 않을 이유가 있을까 하는겁니다.

이에 대한 반론은 세가지 같습니다.

• 그렇게까지 기술을 발전시키기 쉽지 않다.
• 사람 모델링도 안되어 있으니, 종합적 인지모델 자체를 만들기 어렵다
• 어쨌든 사람 가르치듯 가르치면 된다
결국 AI 학습은 양육개념이라고 보면 더 잘 이해됩니다. 사람 아이도 잘 가르치기 어려운데, AI라고 쉽게 가르칠수 있겠습니까. 양자컴퓨팅을 만든 도이치는 진정한 AGI는 체스를 매번 이기도록 만든 때가 아니라 체스를 이기지 않기로 결정하는 능력이 생길 때라고 봅니다.

여기에 따라 나오는 개념이 자유의지입니다. 보상과 처벌을 내면화한 결과라고 생각할 수 있습니다. 자유의지는 윤리와 맞닿아 있고요. 


결국 울프람 말처럼, 인간 고유의 능력은 목적(purpose)입니다. 목적을 가질 때 가치가 생기고 동기가 생깁니다 목적은 희소성의 바다에서 생기는 목마름 같은 결핍일 겁니다. 이걸 인공지능이 어떻게 내포할지가 지켜볼 포인트 같습니다.

다 읽고난 제 느낌은 사실 위너의 말은 시적이거나 은유적으로 보는게 맞겠습니다. 다만 위너는 메기입니다. 그의 상상 덕에 우린 다양한 걱정을 

In [19]:
final_response_template = """
        다음 question에 대해 context에 기반해서 답변해줘. 
        단, 'context에 기반한 ~'과 같은 표현은 사용하지 마.
        question:
        {question}
        contexts:
        {contexts}
        """

final_prompt = PromptTemplate.from_template(final_response_template)

print(final_prompt.format(question=question, contexts=contexts))


        다음 question에 대해 context에 기반해서 답변해줘. 
        단, 'context에 기반한 ~'과 같은 표현은 사용하지 마.
        question:
        AI의 가치 정렬(value alignment) 문제는 구체적으로 어떤 우려를 말하나요?
        contexts:
        [Document(metadata={'title': '인공지능은 무엇이 되려 하는가', 'keywords': '지식인|인류|담론'}, page_content='AGI 문제는 악의를 염려하는게 아니라 그 성능이다. 공포장사가 아니라 안전공학 차원에서 제한을 미리 넣어두자(테그마크)\n\nAI가 걱정되지만 잘 가르치면 괜찮을지도 파\n\n기계보상에 답이 있다 (러셀)\n\n대략 이런데, 위너의 생각을 계승한 학자들이 우려하는 건 가치정렬(value alignment)입니다.\n\n첫째는 원숭이 발 문제 또는 미다스의 손 문제입니다. 시키는 것만 문자그대로 완수하여, 부작용이 문제가 되는 상황입니다. 암치료제를 찾는다고 인류를 대상으로 생체실험을 해버린다든지, 바다 산성을 낮추는 대신 대기를 망쳐놓는다든지입니다.\n\n둘째는 영화적 공포입니다. 더 심화해서 AI가 자체의 욕망대로 움직이는 경우지요. 종이 클립 이슈라고 불립니다. 만일 종이 클립이 AI 기계 시스템의 입맛에 맞다고 치면, 문물을 관장하는 AI서버가 온 바다를 종이클립으로 메우지 않을 이유가 있을까 하는겁니다.\n\n이에 대한 반론은 세가지 같습니다.\n\n• 그렇게까지 기술을 발전시키기 쉽지 않다.\n• 사람 모델링도 안되어 있으니, 종합적 인지모델 자체를 만들기 어렵다\n• 어쨌든 사람 가르치듯 가르치면 된다\n결국 AI 학습은 양육개념이라고 보면 더 잘 이해됩니다. 사람 아이도 잘 가르치기 어려운데, AI라고 쉽게 가르칠수 있겠습니까. 양자컴퓨팅을 만든 도이치는 진정한 AGI는 체스를 매번 이기도록 만든 때가 아니라 체스를 이기지 않기로 결정하는 능력이 생길 

In [20]:
# 최종 답변 출력
final_response = client.chat(
    model="hf.co/LGAI-EXAONE/EXAONE-4.0-1.2B-GGUF:Q4_K_M",
    messages=[
        {
            'role': 'user',
            'content': final_prompt.format(question=question, contexts=contexts),
        },
    ]
)

response = final_response['message']['content']

print(response)

AI의 가치 정렬(value alignment) 문제는 다음과 같은 우려를 말합니다:  
1. **원숭이 발 문제**: 인간 통제 없이 AI가 원하는 행동을 달성할 경우 예상치 못한 부작용 발생 가능성  
2. **자기확대적 욕망**: 기술 발전 과정에서 시스템이 자원 소모나 환경 훼손 등 새로운 위험에 노출될 수 있음(종이클립 예시)  
3. 사용자 교육의 한계: 기존 지식기반 학습 모델로는 복잡한 윤리체계를 효과적으로 주입하기 어려움
